# AlphaFold metrics
Created by Andreas 04.04.2025

This is the main notebook to add the following informations and metrics to a given AlphaFold run:

1. [RMSD](#3-rmsd-with-pymol)
2. [DockQ](#4-dockq)
3. [ipSAE](#5-ipsae-metric)
4. [Interface Interaction metrics](#6-interaction-metrics)

The sections (e.g. RMSD, DockQ) can be run individually to recalculate only some metrics. For this, set the *load_previous* setting to True.

The benchmark set of Lee et al. encodes information about a PPI in a string (e.g. *PF10208_PF00012_6H9U_B_resi130_resi169.A_resi30_resi406*). This script breaks it down into the PDB ID, PFAM ID, and ELM instance. Also, as it was worked with minimal interacting regions (i.e. only the motif or domain), the included chains (i.e. their IDs) as well as the boundaries were either extracted from the prediction name (DDI) or determined by comparing with the experimentally solved structure.

As input, the .pdb files need to be stored in the corrosponding benchmark set:

```bash
├── DDI
│   ├── known_DDI
│   └── random_DDI
├── DMI
│   ├── known_DMI
│   ├── mutations_DMI
│   └── random_DMI
```

Every structure inside this folder should follow this layout:
```bash
├── DEG_APCC_KENBOX_2_4GGD
│   ├── ranked_0.pdb
│   ├── ranked_1.pdb
│   ├── ranked_2.pdb
│   ├── ranked_3.pdb
│   └── ranked_4.pdb
```

The experimentally solved structures should have the following folder structure:

```bash
├── DDI
├── DDI_hydrogens
├── DMI
├── DMI_hydrogens
```

with the hydrogen folders containing the structures with added hydrogen.

It is also necessary to provide the output file of the AlphaFold run. For AlphaFold 2, this file was created by Lee et al., while for AlphaFold 3 the <a href="AF3 raw output parsing.ipynb">AF3 raw output parsing.ipynb</a> notebook is able to generate one. This file should list all AlphaFold predictions with AlphaFold derived metrics like the model cofidence, the number of clashes etc.

By default, you specify a *resource* path with a *AF2*/*AF3* and *solved* folder inside in it. Also, a *AF3_hydrogens* folder can be included.

### 0 Imports and Settings

In [1]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.axes._axes import Axes
from matplotlib.figure import Figure
from pathlib import Path
from sklearn.metrics import roc_curve, roc_auc_score
import re
import tempfile
import shutil
import os
import subprocess
import sys
stdout, stderr = sys.stdout, sys.stderr
from typing import Literal

import pymol
from Bio.SeqUtils import seq1
from Bio.PDB import PDBParser
from Bio.PDB.Structure import Structure as BioPy_PDBStructure
from Bio.PDB.Model import Model as BioPy_PDBModel
from Bio.PDB.Chain import Chain
from Bio.PDB.PDBExceptions import PDBConstructionException
parser = PDBParser(QUIET=True)

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [2]:
# Settings

# Which AF output should be parsed
af_mode: Literal["AF2", "AF3"] = "AF3"
# Should be the hydrogens folder used?
af3_hydrogens: bool = True

# Path to resource folder with the structures and metadata tables
path_resources = Path("/Users/imb/Desktop")
# Path to the Luck Drive folder (used for ipSAE metric to get the json file)
path_AF_luck_drive = Path("/Volumes/imb-luckgr/imb-luckgr2/projects/AlphaFold")
if af_mode == "AF3":
    path_AF_luck_drive = path_AF_luck_drive / "AlphaFold3"

# Paths to the local folders
path_AF = path_resources / ((af_mode + "_hydrogens") if (af3_hydrogens and af_mode == "AF3") else af_mode)
path_solved = path_resources / "DMI_solved_structures"

# The path to the ipsae.py
path_ipsae_script = Path("/Users/imb/Desktop/ipsae.py")

# If set to true, load the previous dataframe
load_previous = False

# To parse structures, Pymol should be run headless. However, for debugging the code the GUI may be helpful
pymol_headless = True

In [3]:
if not pymol_headless:
    pymol.finish_launching()
    sys.stdout = stdout # Needed in case of debugging to redicrct 
    sys.stderr = stderr

In [4]:
def enhance_dataframe():
    """ Reorder the columns in the dataAF dataframe and convert to an appropriate dtype """
    global dataAF
    # Some columns are integer, but contain None values. Default behaviour of pandas is to use dtype float. Therefore change the dtype to the pd.Int64Dtype allowing None
    for c in ["chainA_start", "chainA_end", "chainB_start", "chainB_end", "num_mutations", "num_align_atoms_domain", "num_align_resi_domain", "hbonds", "salt_bridges", "hydrophobic_interactions", "disulfide_bonds"]:
        if c not in dataAF.columns:
            print(f"Column {bcolors.FAIL}{c}{bcolors.ENDC} not (yet) in data frame")
            continue
        dataAF[c] = dataAF[c].astype(pd.Int64Dtype())

    def _reorder_column(c:list[str], column: str, prev_column: str = None, index:int = None):
        if column not in c:
            print(f"Column {bcolors.FAIL}{column}{bcolors.ENDC} not (yet) in data frame")
            return
        if index is None:
            if prev_column not in c:
                print(f"Column {bcolors.FAIL}{prev_column}{bcolors.ENDC} (used for sorting) not (yet) in data frame")
                return
            index = c.index(prev_column) + 1
        c.remove(column)
        c.insert(index, column)

    # Reordering of the columns
    c = list(dataAF.columns)
    if af_mode == "AF2":
        _reorder_column(c, "run_id", index=1)
        _reorder_column(c, "benchmark_set", index=2)
    elif af_mode == "AF3":
        _reorder_column(c, "benchmark_set", index=1)
    _reorder_column(c, "prediction_name", prev_column="benchmark_set")
    _reorder_column(c, "model_id", prev_column="prediction_name")
    _reorder_column(c, "model_path", prev_column="model_id")  # Added line
    if af_mode == "AF3": 
        _reorder_column(c, "ranking_score", prev_column="model_path")  # Updated 
        _reorder_column(c, "chainA_length", prev_column="ranking_score")
    else:
        _reorder_column(c, "chainA_length", prev_column="model_path")  # Updated 
    _reorder_column(c, "chainB_length", prev_column="chainA_length")
    _reorder_column(c, "chainA_id", prev_column="chainB_length")
    _reorder_column(c, "chainB_id", prev_column="chainA_id")
    _reorder_column(c, "chainA_start", prev_column="chainB_id")
    _reorder_column(c, "chainA_end", prev_column="chainA_start")
    _reorder_column(c, "chainB_start", prev_column="chainA_end")
    _reorder_column(c, "chainB_end", prev_column="chainB_start")
    _reorder_column(c, "PDB_id", prev_column="chainB_end")
    _reorder_column(c, "ELM_instance", prev_column="PDB_id")
    _reorder_column(c, "DDI_pfam_id", prev_column="ELM_instance")
    _reorder_column(c, "PDB_id_random_paired", prev_column="DDI_pfam_id")
    _reorder_column(c, "ELM_instance_random_paired", prev_column="PDB_id_random_paired")
    _reorder_column(c, "DDI_pfam_id_random_paired", prev_column="ELM_instance_random_paired")
    _reorder_column(c, "sequence_initial", prev_column="DDI_pfam_id_random_paired")
    _reorder_column(c, "sequence_mutated", prev_column="sequence_initial")
    _reorder_column(c, "num_mutations", prev_column="sequence_mutated")

    _reorder_column(c, "align_score_domain", prev_column="num_atom_atom_contact")
    _reorder_column(c, "num_align_atoms_domain", prev_column="align_score_domain")
    _reorder_column(c, "num_align_resi_domain", prev_column="num_align_atoms_domain")
    _reorder_column(c, "RMSD_domain", prev_column="num_align_resi_domain")
    _reorder_column(c, "RMSD_backbone_peptide", prev_column="RMSD_domain")
    _reorder_column(c, "RMSD_all_atom_peptide", prev_column="RMSD_backbone_peptide")
    _reorder_column(c, "RMSD_all_atom", prev_column="RMSD_all_atom_peptide")

    _reorder_column(c, "buried_area", prev_column="Fnonnat")
    _reorder_column(c, "min_distance", prev_column="buried_area")
    _reorder_column(c, "disulfide_bonds", prev_column="min_distance")
    _reorder_column(c, "salt_bridges", prev_column="disulfide_bonds")
    _reorder_column(c, "hbonds", prev_column="salt_bridges")
    _reorder_column(c, "hydrophobic_interactions", prev_column="hbonds")
    

    dataAF = dataAF[c]
    

In [5]:
# Load data
if load_previous:
    dataAF = pd.read_csv(path_resources / "AF3_output_only_known_minimal.tsv" , sep="\t")
else:
    # Read in the AF data
    if af_mode == "AF2":
        dataAF = pd.read_excel(path_resources / "AF3_output_only_known_minimal.xlsx")

        print(dataAF.columns.tolist())

        # Drop columns to recalculate them
        dataAF.drop(columns=["RMSD_domain", "num_align_atoms_domain", "align_score_domain", "num_align_resi_domain", "RMSD_backbone_peptide", "RMSD_all_atom_peptide", "known_motif_plddt", "DockQ", "iRMS", "LRMS", "Fnonnat", "label"], inplace=True)


        # Adding benchmark set column
        benchmark_set_replace_dict = {"1": "mutations_DMI", "2" : "mutations_DMI", "approved minimal DDI": "known_DDI", "known minimal": "known_DMI", "random minimal": "random_minimal", "random minimal DDI": "random_DDI", "known_extension" : "known_extension", "random_extension" : "random_extension"}
        dataAF["benchmark_set"] = None
        dataAF["num_mutations"] = None

        for i, row in dataAF.iterrows():
            if row["mutation_in_motif"] == "1":
                dataAF.at[i, "num_mutations"] = 1
            elif row["mutation_in_motif"] == "2":
                dataAF.at[i, "num_mutations"] = 2
            benchmark_set = benchmark_set_replace_dict[row["mutation_in_motif"]]
            dataAF.at[i, "benchmark_set"] = benchmark_set
        dataAF.drop(columns=["mutation_in_motif"], inplace=True)

    elif af_mode == "AF3":
        dataAF = pd.read_csv(path_resources / "AF3_output_only_known_minimal.tsv", sep="\t")

        benchmark_set_replace_dict = {"mutations": "mutations_DMI", "known_minimal": "known_DMI", "known_ddi": "known_DDI", "random_minimal": "random_minimal", "random_ddi": "random_DDI", "known_extension" : "known_extension", "random_extension" : "random_extension"}
            
        for i, row in dataAF.iterrows():
            benchmark_set = benchmark_set_replace_dict[row["benchmark_set"]]
            dataAF.at[i, "benchmark_set"] = benchmark_set
enhance_dataframe()
display(dataAF)

Column chainA_start not (yet) in data frame
Column chainA_end not (yet) in data frame
Column chainB_start not (yet) in data frame
Column chainB_end not (yet) in data frame
Column num_mutations not (yet) in data frame
Column num_align_atoms_domain not (yet) in data frame
Column num_align_resi_domain not (yet) in data frame
Column hbonds not (yet) in data frame
Column salt_bridges not (yet) in data frame
Column hydrophobic_interactions not (yet) in data frame
Column disulfide_bonds not (yet) in data frame
Column chainA_id not (yet) in data frame
Column chainB_id not (yet) in data frame
Column chainA_start not (yet) in data frame
Column chainA_end not (yet) in data frame
Column chainB_start not (yet) in data frame
Column chainB_end not (yet) in data frame
Column PDB_id not (yet) in data frame
Column ELM_instance not (yet) in data frame
Column DDI_pfam_id not (yet) in data frame
Column PDB_id_random_paired not (yet) in data frame
Column ELM_instance_random_paired not (yet) in data frame
Co

,model_preset,benchmark_set,prediction_name,model_id,model_path,ranking_score,chainA_length,chainB_length,fraction_disordered,has_clash,...,chainA_intf_avg_plddt,chainB_intf_avg_plddt,intf_avg_plddt,num_chainA_intf_res,num_chainB_intf_res,num_res_res_contact,num_atom_atom_contact,iPAE,pDockQ,chains_flipped
0,alphafold3,random_minimal,MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30,ranked_0,AlphaFold_benchmark_DMI/random_minimal/distrac...,0.65,237,5,0.02,0.0,...,83.95,49.49,74.38,13,5,20,178,10.60,0.07,True
1,alphafold3,random_minimal,MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30,ranked_1,AlphaFold_benchmark_DMI/random_minimal/distrac...,0.65,237,5,0.02,0.0,...,84.70,48.41,74.62,13,5,21,224,11.00,0.09,True
2,alphafold3,random_minimal,MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30,ranked_2,AlphaFold_benchmark_DMI/random_minimal/distrac...,0.65,237,5,0.02,0.0,...,85.32,49.80,75.97,14,5,21,196,10.80,0.08,True
3,alphafold3,random_minimal,MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30,ranked_3,AlphaFold_benchmark_DMI/random_minimal/distrac...,0.64,237,5,0.02,0.0,...,84.09,50.50,75.69,15,5,22,199,11.80,0.08,True
4,alphafold3,random_minimal,MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30,ranked_4,AlphaFold_benchmark_DMI/random_minimal/distrac...,0.62,237,5,0.02,0.0,...,83.69,48.32,73.87,13,5,21,206,11.90,0.08,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,alphafold3,random_minimal,MTRG_PTS1_2C0L.DLIG_WD40_WDR5_WIN_2_4CY3,ranked_0,AlphaFold_benchmark_DMI/random_minimal/gloomy_...,0.93,312,4,0.02,0.0,...,96.90,84.04,94.45,17,4,26,239,3.50,0.31,True
676,alphafold3,random_minimal,MTRG_PTS1_2C0L.DLIG_WD40_WDR5_WIN_2_4CY3,ranked_1,AlphaFold_benchmark_DMI/random_minimal/gloomy_...,0.92,312,4,0.01,0.0,...,97.70,83.70,95.04,17,4,26,233,3.60,0.31,True
677,alphafold3,random_minimal,MTRG_PTS1_2C0L.DLIG_WD40_WDR5_WIN_2_4CY3,ranked_2,AlphaFold_benchmark_DMI/random_minimal/gloomy_...,0.92,312,4,0.02,0.0,...,96.88,83.24,94.40,18,4,28,246,3.65,0.30,True
678,alphafold3,random_minimal,MTRG_PTS1_2C0L.DLIG_WD40_WDR5_WIN_2_4CY3,ranked_3,AlphaFold_benchmark_DMI/random_minimal/gloomy_...,0.91,312,4,0.01,0.0,...,97.64,83.07,94.86,17,4,26,232,3.60,0.30,True


In [6]:
# Read in solved structure data

dataSolved = pd.DataFrame(columns=["set", "PDB_id", "DDI_pfam_id", "path", "chainA_id", "chainB_id"])

# DMI
for structure_file in [p for p in path_solved.iterdir() if p.is_file() and p.suffix == ".pdb"]:
    pdb_id = structure_file.name.split("_")[0]
    dataSolved.loc[len(dataSolved)] = {"set" : "DMI", "PDB_id": pdb_id, "path": structure_file.relative_to(path_solved), "chainA_id": "A", "chainB_id": "B"}

# DDI
"""
for structure_file in [p for p in path_solved.iterdir() if p.is_file() and p.suffix == ".pdb"]:
    ddi_pfam_id = "_".join(structure_file.name.split("_")[0:2])
    pdb_id = structure_file.name.split("_")[2]
    chainA_id = structure_file.name.split("_")[3][0]
    chainB_id = structure_file.name.split("_")[3][1]
    dataSolved.loc[len(dataSolved)] = {"set" : "DDI", "PDB_id": pdb_id, "DDI_pfam_id": ddi_pfam_id, "path": structure_file.relative_to(path_solved), "chainA_id": chainA_id, "chainB_id": chainB_id}
"""
display(dataSolved)

,set,PDB_id,DDI_pfam_id,path,chainA_id,chainB_id
0,DMI,2JK9,NaN,2JK9_min_DMI.pdb,A,B
1,DMI,1B8Q,NaN,1B8Q_min_DMI.pdb,A,B
2,DMI,4GGD,NaN,4GGD_min_DMI.pdb,A,B
3,DMI,5F74,NaN,5F74_min_DMI.pdb,A,B
4,DMI,1DDV,NaN,1DDV_min_DMI.pdb,A,B
...,...,...,...,...,...,...
133,DMI,1UTC,NaN,1UTC_min_DMI.pdb,A,B
134,DMI,3GM1,NaN,3GM1_min_DMI.pdb,A,B
135,DMI,1NTV,NaN,1NTV_min_DMI.pdb,A,B
136,DMI,1ZUB,NaN,1ZUB_min_DMI.pdb,A,B


### 1 Parsing the file names
Many informations (PDB ID, mutation sequence, ...) are included in the filename. This section parses them and adds them to the metrics data frame. The detected values include:
* **PDB_id** (all structures): RCSB Protein Data Bank ID
* **ELM_instance** (DMI): ID of the motif in the Eukaryotic Linear Motif database
* **DDI_pfam_id** (DDI): ID of the two domains in the Pfam Database separated by an underscore
* **PDB_id_random_paired**, **ELM_instance_random_paired** and **DDI_pfam_id_random_paired** (random_DMI, random_DDI): The id of the second domain/motif for the randomly paired DMI and DDI. For DMI, the random fields always encode the motif.
* **sequence_initial** and **sequence_mutated** (mutations_DMI): For the mutations, this fields encode the initinal and mutated sequence of the motif
* **chainA_id**, **chainA_start**, **chainA_end** and the same three for **chainB** (all structures where data is available): The start and end residue as well as the ID of the chains in the experimentally solved structure. For known_DDI, the information is encoded in the prediction name and carried over to the random_DDI set. For DMI structures, the information will be added in section 2 by aligning with the template file. Chain A and B refer to the IDs in the AlphaFold predictions. For DMI, chain B is always the motif chain.

Note: known_extensions were excluded from the benchmark set, but if you need to parse them remove the comments in the code cell below. But the code needs to be tested !!

In [7]:
# Regex checks on filename
regex_paired_DMI = r"^([\w\-]+)_(\w{4})$"
regex_random_minimal = r"^M([\w\-]+)_(\w{4})\.D([\w\-]+)_(\w{4})$"
regex_mutated_DMI = r"^([\w\-]+)_(\w{4})_(\w+)\.([A-Za-z]+)$"
regex_known_extension_DMI = r"^([\w-]+)_((Mmin)|(MFL)|(M[\d]+_M[\d]+))_((DFL)|(Dmin)|(D[\d]+_D[\d]+))$"
regex_ddi_known = r"^([^\W_]+_[^\W_]+)_(\w{4})_(\w+)_resi(\d+)_resi(\d+).(\w+)_resi(\d+)_resi(\d+)$"
regex_ddi_random = r"^D1([^\W_]+_[^\W_]+)_(\w{4}).D2([^\W_]+_[^\W_]+)_(\w{4})$"


dataAF["PDB_id"] = None
dataAF["ELM_instance"] = None
dataAF["DDI_pfam_id"] = None
dataAF["PDB_id_random_paired"] = None
dataAF["ELM_instance_random_paired"] = None
dataAF["DDI_pfam_id_random_paired"] = None
dataAF["sequence_initial"] = None
dataAF["sequence_mutated"] = None
# known_extensions have not been run. Therefore exclude them here but keep the code for them
#dataAF["known_extension_motif"] = None 
#dataAF["known_extension_domain"] = None
dataAF["chainA_id"] = None
dataAF["chainB_id"] = None
dataAF["chainA_start"] = None
dataAF["chainA_end"] = None
dataAF["chainB_start"] = None
dataAF["chainB_end"] = None

for i, row in dataAF.iterrows():
    pdb_id, pdb_id_2, elm_instance, elm_instance_2, sequence, sequence_f = None, None, None, None, None, None
    known_extensionM, known_extensionD, chain1_letter, chain2_letter, ddi_pfam_id, ddi_pfam_id_random_paired = None, None, None, None, None, None
    c1_start, c1_end, c2_start, c2_end = None, None, None, None
    if (benchmark_set := row["benchmark_set"]) == "known_DMI":
        if (r1 := re.search(regex_paired_DMI, row["prediction_name"])) is not None and len(r1.groups()) == 2:
            elm_instance = r1.groups()[0]
            pdb_id = r1.groups()[1]
            chain1_letter, chain2_letter = "A", "B"
    elif benchmark_set == "random_minimal":
        if (r := re.search(regex_random_minimal, row["prediction_name"])) is not None and len(r.groups()) == 4:
            # Contraintuitive, but here before dot is motif and after dot is domain
            elm_instance_2 = r.groups()[0]
            pdb_id_2 = r.groups()[1]
            elm_instance = r.groups()[2]
            pdb_id = r.groups()[3]
            chain1_letter, chain2_letter = "A", "B"
    elif benchmark_set == "mutations_DMI":
        if (r := re.search(regex_mutated_DMI, row["prediction_name"])) is not None and len(r.groups()) == 4:
            elm_instance = r.groups()[0]
            pdb_id = r.groups()[1]
            sequence = r.groups()[2]
            sequence_f = r.groups()[3]
            chain1_letter, chain2_letter = "A", "B"

            if af_mode == "AF3":
                assert len(sequence) == len(sequence_f)
                num_mutations = 0
                for n in range(len(sequence)):
                    if sequence[n] != sequence_f[n]: 
                        num_mutations += 1
                dataAF.at[i, "num_mutations"] = num_mutations
    #elif benchmark_set == "known_extension":
    #    if (r := re.search(regex_known_extension_DMI, row["prediction_name"])) is not None and len(r.groups()) == 9:
    #        elm_instance = r.groups()[0]
    #        known_extensionM = r.groups()[1]
    #        known_extensionD = r.groups()[5]
    elif benchmark_set == "known_DDI":
        if (r := re.search(regex_ddi_known, row["prediction_name"])) is not None and len(r.groups()) == 8:
            ddi_pfam_id = r.groups()[0]
            pdb_id = r.groups()[1]
            chain1_letter = r.groups()[2]
            c1_start = r.groups()[3]
            c1_end = r.groups()[4]
            chain2_letter = r.groups()[5]
            c2_start = r.groups()[6]
            c2_end = r.groups()[7]

            if pdb_id != pdb_id.upper():
                pdb_id = pdb_id.upper()
                new_prediction_name = row["prediction_name"][:r.span(2)[0]] + pdb_id + row["prediction_name"][r.span(2)[1]:]
                print(f"Fixed prediction_name in set {benchmark_set} from {row['prediction_name']} to {new_prediction_name}")
                dataAF.at[i, "prediction_name"] = new_prediction_name
    elif benchmark_set == "random_DDI":
        if (r := re.search(regex_ddi_random, row["prediction_name"])) is not None and len(r.groups()) == 4:
            ddi_pfam_id = r.groups()[0]
            pdb_id = r.groups()[1]   
            ddi_pfam_id_random_paired = r.groups()[2]
            pdb_id_2 = r.groups()[3]  
    else:
        raise RuntimeError(f"Regex failed on {row['prediction_name']}")
    
    dataAF.at[i, "PDB_id"] =  pdb_id
    dataAF.at[i, "PDB_id_random_paired"] =  pdb_id_2
    dataAF.at[i, "ELM_instance"] =  elm_instance
    dataAF.at[i, "ELM_instance_random_paired"] =  elm_instance_2
    dataAF.at[i, "sequence_initial"] =  sequence
    dataAF.at[i, "sequence_mutated"] =  sequence_f
    #dataAF.at[i, "known_extension_motif"] =  known_extensionM
    #dataAF.at[i, "known_extension_domain"] =  known_extensionD
    dataAF.at[i, "chainA_id"] =  chain1_letter
    dataAF.at[i, "chainB_id"] =  chain2_letter
    dataAF.at[i, "DDI_pfam_id"] =  ddi_pfam_id
    dataAF.at[i, "DDI_pfam_id_random_paired"] =  ddi_pfam_id_random_paired
    dataAF.at[i, "chainA_start"] =  int(c1_start) if c1_start is not None else None
    dataAF.at[i, "chainA_end"] =  int(c1_end) if c1_end is not None else None
    dataAF.at[i, "chainB_start"] =  int(c2_start) if c2_start is not None else None
    dataAF.at[i, "chainB_end"] =  int(c2_end) if c2_end is not None else None

# The chain ids as well as start and end residues for the random DDI can be obtained from the known DDI
for i, row in dataAF[dataAF["benchmark_set"] == "random_DDI"].iterrows():
    prediction_name = row["prediction_name"]
    pdb_id, pdb_id_2 = row["PDB_id"], row["PDB_id_random_paired"]
    ddi_pfam_id, ddi_pfam_id_2 = row["DDI_pfam_id"], row["DDI_pfam_id_random_paired"]
    
    if len(list((_row1 := dataAF[np.logical_and(dataAF["benchmark_set"] == "known_DDI", np.logical_and(dataAF["PDB_id"] == pdb_id, dataAF["DDI_pfam_id"] == ddi_pfam_id))])["chainA_id"])) == 0:
        print(f"Can't find {pdb_id} from {prediction_name} (random_ddi, chain A) in the known_DDI set")
        continue
    if len(list((_row2 := dataAF[np.logical_and(dataAF["benchmark_set"] == "known_DDI", np.logical_and(dataAF["PDB_id"] == pdb_id_2, dataAF["DDI_pfam_id"] == ddi_pfam_id_2))])["chainB_id"])) == 0:
        print(f"Can't find {pdb_id_2} from {prediction_name} (random_ddi, chain B) in the known_DDI set")
        continue
    dataAF.at[i, "chainA_id"] = list(_row1["chainA_id"])[0]
    dataAF.at[i, "chainA_start"] = list(_row1["chainA_start"])[0]
    dataAF.at[i, "chainA_end"] = list(_row1["chainA_end"])[0]
    dataAF.at[i, "chainB_id"] =  list(_row2["chainB_id"])[0]
    dataAF.at[i, "chainB_start"] = list(_row2["chainB_start"])[0]
    dataAF.at[i, "chainB_end"] = list(_row2["chainB_end"])[0]
print("\n", f"Rows, where the regex failed")
display(dataAF[dataAF["PDB_id"].isna()])
display(dataAF)


 Rows, where the regex failed


,model_preset,benchmark_set,prediction_name,model_id,model_path,ranking_score,chainA_length,chainB_length,fraction_disordered,has_clash,...,ELM_instance_random_paired,DDI_pfam_id_random_paired,sequence_initial,sequence_mutated,chainA_id,chainB_id,chainA_start,chainA_end,chainB_start,chainB_end


,model_preset,benchmark_set,prediction_name,model_id,model_path,ranking_score,chainA_length,chainB_length,fraction_disordered,has_clash,...,ELM_instance_random_paired,DDI_pfam_id_random_paired,sequence_initial,sequence_mutated,chainA_id,chainB_id,chainA_start,chainA_end,chainB_start,chainB_end
0,alphafold3,random_minimal,MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30,ranked_0,AlphaFold_benchmark_DMI/random_minimal/distrac...,0.65,237,5,0.02,0.0,...,DEG_APCC_KENBOX_2,None,None,None,A,B,None,None,None,None
1,alphafold3,random_minimal,MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30,ranked_1,AlphaFold_benchmark_DMI/random_minimal/distrac...,0.65,237,5,0.02,0.0,...,DEG_APCC_KENBOX_2,None,None,None,A,B,None,None,None,None
2,alphafold3,random_minimal,MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30,ranked_2,AlphaFold_benchmark_DMI/random_minimal/distrac...,0.65,237,5,0.02,0.0,...,DEG_APCC_KENBOX_2,None,None,None,A,B,None,None,None,None
3,alphafold3,random_minimal,MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30,ranked_3,AlphaFold_benchmark_DMI/random_minimal/distrac...,0.64,237,5,0.02,0.0,...,DEG_APCC_KENBOX_2,None,None,None,A,B,None,None,None,None
4,alphafold3,random_minimal,MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30,ranked_4,AlphaFold_benchmark_DMI/random_minimal/distrac...,0.62,237,5,0.02,0.0,...,DEG_APCC_KENBOX_2,None,None,None,A,B,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,alphafold3,random_minimal,MTRG_PTS1_2C0L.DLIG_WD40_WDR5_WIN_2_4CY3,ranked_0,AlphaFold_benchmark_DMI/random_minimal/gloomy_...,0.93,312,4,0.02,0.0,...,TRG_PTS1,None,None,None,A,B,None,None,None,None
676,alphafold3,random_minimal,MTRG_PTS1_2C0L.DLIG_WD40_WDR5_WIN_2_4CY3,ranked_1,AlphaFold_benchmark_DMI/random_minimal/gloomy_...,0.92,312,4,0.01,0.0,...,TRG_PTS1,None,None,None,A,B,None,None,None,None
677,alphafold3,random_minimal,MTRG_PTS1_2C0L.DLIG_WD40_WDR5_WIN_2_4CY3,ranked_2,AlphaFold_benchmark_DMI/random_minimal/gloomy_...,0.92,312,4,0.02,0.0,...,TRG_PTS1,None,None,None,A,B,None,None,None,None
678,alphafold3,random_minimal,MTRG_PTS1_2C0L.DLIG_WD40_WDR5_WIN_2_4CY3,ranked_3,AlphaFold_benchmark_DMI/random_minimal/gloomy_...,0.91,312,4,0.01,0.0,...,TRG_PTS1,None,None,None,A,B,None,None,None,None


### 2 Adding domain and motif start / end from template file
While for the DDI structures selection start and end are included in the filename, for DMI structures there is absolutely no information about start/end of motif and domain. At least, the DMI structures are cut to only include the minimal domain/motif, but there still may be mutations or missing residues in experimental structures. To restore this information use the template and perform a simple search for three consecutive residues in both chains and calculate the offset between the chain IDs. Then take the most common offset and use it if at least 50 % of the AF residues were matched this way.

In [ ]:
def align_sequences(chain_af:  Chain, chain_template: Chain) -> tuple[int, int, float, str, str]:
    """ Estimate the residue id offset between two chains based on a neighbour local alignment (BioPython has no convinient alignment function).
    
        :returns tuple[int, int, float, str, str]: Start ID, End ID, score, Sequence Chain A, Sequence Chain B
    """
    residues_af = [r for r in chain_af.get_residues()]
    residues_tpl = [r for r in chain_template.get_residues()]
    seq_af = seq1(''.join([r.resname for r in residues_af]))
    seq_tpl = seq1(''.join([r.resname for r in residues_tpl]))
    offset_list = []

    misscounts = 0
    for a0, a1, a2 in zip(residues_af[:-2], residues_af[1:-1], residues_af[2:]):
        _found = False
        for t0, t1, t2 in zip(residues_tpl[:-2], residues_tpl[1:-1], residues_tpl[2:]):
            if a0.resname == t0.resname and a1.resname == t1.resname and a2.resname == t2.resname:
                offset_list.append(t1.id[1] - a1.id[1])
                _found = True
        if not _found:
            misscounts += 1

    # For degenerated short chains (motif) use no neighbours for matching
    # if len(offset_list) == 0:
    #     for r1 in residues_af:
    #         for r2 in residues_tpl:
    #             if r1.resname == r2.resname:
    #                 offset_list.append(r2.id[1] - r1.id[1])

    if len(offset_list) == 0:
        return (None, None, 0, seq_af, seq_tpl)
    offsets, counts = np.unique(offset_list, return_counts=True)
    offset = offsets[np.argmax(counts)]
    score = 1 - misscounts/(len(residues_af) - 2)
    return  offset + 1, offset + len(residues_af), score, seq_af, seq_tpl

for i, row in dataAF[dataAF["benchmark_set"].isin(["random_minimal"])].iterrows():
    pdb_id = str(row["PDB_id"])
    pdb_id_2 = None
    if row["PDB_id_random_paired"] is not None:
        pdb_id_2 = str(row["PDB_id_random_paired"])
    prediction_name = row["prediction_name"]
    benchmark_set = row["benchmark_set"]
    model_id = row["model_id"]

    if model_id == "ranked_0":
        print(bcolors.OKBLUE + f"{prediction_name} ({benchmark_set})" + bcolors.ENDC)

    #if not prediction_name == "MLIG_MYND_2_2ODD.DMOD_SUMO_for_1_1KPS": continue

    af_path = path_resources / "DMI" / benchmark_set / prediction_name / (model_id + ".pdb")
    af_biopy = parser.get_structure("structure", file=af_path)[0]
    chainA_af = af_biopy["A"]
    chainB_af = af_biopy["B"]    

    template1_path = path_solved / (pdb_id + "_min_DMI.pdb")
    if not template1_path.exists():
        print(f"\t", bcolors.WARNING + f"{prediction_name} has no template file for {pdb_id}" + bcolors.WARNING)
        continue
    template1_biopy = parser.get_structure("structure", file=template1_path)[0]
    chainA_tlp = template1_biopy["A"]
    if pdb_id_2 is not None:
        template2_path = path_solved / (pdb_id_2 + "_min_DMI.pdb")
        if not template2_path.exists():
            print(f"\t", f"{prediction_name} has no template file for {pdb_id}")
            continue
        template2_biopy = parser.get_structure("structure", file=template2_path)[0]
        chainB_tlp = template2_biopy["B"]
    else:
        chainB_tlp = template1_biopy["B"]

    chainA_start, chainA_end, chainA_score, seqA_af, seqA_tpl = align_sequences(chain_af=chainA_af, chain_template=chainA_tlp)
    if chainA_start is not None:
        if model_id == "ranked_0":
            print("\t", f"chainA: {chainA_start}-{chainA_end} ({bcolors.WARNING if chainA_score < 0.5 else ''}{chainA_score:0.3f}{bcolors.ENDC})")
        dataAF.at[i, "chainA_start"] =  chainA_start
        dataAF.at[i, "chainA_end"] =  chainA_end
    else:
        if model_id == "ranked_0":
            print(f"\t", bcolors.WARNING + "Chain A alignment failed" + bcolors.ENDC)
    if model_id == "ranked_0" and chainA_score < 0.5:
        print("\t\t", seqA_af)
        print("\t\t", seqA_tpl)

    chainB_start, chainB_end, chainB_score, seqB_af, seqB_tpl = align_sequences(chain_af=chainB_af, chain_template=chainB_tlp)
    if chainB_start is not None:
        if model_id == "ranked_0":
            print("\t", f"chainB: {chainB_start}-{chainB_end} ({bcolors.WARNING if chainB_score < 0.5 else ''}{chainB_score:0.3f}{bcolors.ENDC})")
        dataAF.at[i, "chainB_start"] =  chainB_start
        dataAF.at[i, "chainB_end"] =  chainB_end
    else:
        if model_id == "ranked_0":
            print(f"\t", bcolors.WARNING + "Chain B alignment failed" + bcolors.ENDC)
    if model_id == "ranked_0" and chainB_score < 0.5:
        print("\t\t", seqB_af)
        print("\t\t", seqB_tpl)
    
    

# For the mutations, the alignment mostly fails. For those restore the information using the known_DMI dataset
for i, row in dataAF[dataAF["benchmark_set"].isin(["mutations_DMI"])].iterrows():
    prediction_name = row["prediction_name"]
    benchmark_set = row["benchmark_set"]
    pdb_id = row["PDB_id"]
    pdb_id_2 = row["PDB_id_random_paired"] if row["PDB_id_random_paired"] is not None else pdb_id
    
    if len(list((_row1 := dataAF[np.logical_and(dataAF["benchmark_set"] == "known_DMI", dataAF["PDB_id"] == pdb_id)])["chainA_id"])) == 0:
        if model_id == "ranked_0":
            print(f"Can't find {pdb_id} from {prediction_name} ({benchmark_set}, chain A) in the known_DMI set")
        continue
    if len(list((_row2 := dataAF[np.logical_and(dataAF["benchmark_set"] == "known_DMI", dataAF["PDB_id"] == pdb_id_2)])["chainB_id"])) == 0:
        if model_id == "ranked_0":
            print(f"Can't find {pdb_id_2} from {prediction_name} ({benchmark_set}, chain B) in the known_DMI set")
        continue
    dataAF.at[i, "chainA_start"] = list(_row1["chainA_start"])[0]
    dataAF.at[i, "chainA_end"] = list(_row1["chainA_end"])[0]
    dataAF.at[i, "chainB_start"] = list(_row2["chainB_start"])[0]
    dataAF.at[i, "chainB_end"] = list(_row2["chainB_end"])[0]

MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30 (random_minimal)
	 Chain A alignment failed
		 SKENV
		 GGYVAPKAVWLPAVKAKGLEISGTFTHRQGHIYMEMNFTNKALQHMTDFAIQFNKNSFGVIPSTPLAIHTPLMPNQSIDVSLPLNTLGPVMKMEPLNNLQVAVKNNIDVFYFSCLIPLNVLFVEDGKMERQVFLATWKDIPNENELQFQIKECHLNADTVSSKLQNNNVYTIAKRNVEGQDMLYQSLKLTNGIWILAELRIQPGNPNYTLSLKCRAPEVSQYIYQVYDSILKN
	 Chain B alignment failed
		 GMAPGGYVAPKAVWLPAVKAKGLEISGTFTHRQGHIYMEMNFTNKALQHMTDFAIQFNKNSFGVIPSTPLAIHTPLMPNQSIDVSLPLNTLGPVMKMEPLNNLQVAVKNNIDVFYFSCLIPLNVLFVEDGKMERQVFLATWKDIPNENELQFQIKECHLNADTVSSKLQNNNVYTIAKRNVEGQDMLYQSLKLTNGIWILAELRIQPGNPNYTLSLKCRAPEVSQYIYQVYDSILKN
		 SKENV
MDEG_COP1_1_5IGO.DDOC_USP7_MATH_1_3MQS (random_minimal)
	 Chain A alignment failed
		 SDQIVPEY
		 TSWRSEATFQFTVERFSRLSESVLSPPCFVRNLPWKIMVMPRFQKSVGFFLQCNAESDSTSWSCHAQAVLKIINYRDDEKSFSRRISHLFFHKENDWGFSNFMAWSEVTDPEKGFIDDDKVTFEVFVQADAPHGVAW
	 Chain B alignment failed
		 TSWRSEATFQFTVERFSRLSESVLSPPCFVRNLPWKIMVMPRFYPDRPHQKSVGFFLQCNAESDSTSWSCHAQAVLKIINYRDDEKSFSRRISHLFFHKENDWGFSNFMAWSEVTDP

In [9]:
dataAF[np.logical_and(dataAF["model_id"] == "ranked_0", True)]


,model_preset,benchmark_set,prediction_name,model_id,model_path,ranking_score,chainA_length,chainB_length,fraction_disordered,has_clash,...,ELM_instance_random_paired,DDI_pfam_id_random_paired,sequence_initial,sequence_mutated,chainA_id,chainB_id,chainA_start,chainA_end,chainB_start,chainB_end
0,alphafold3,random_minimal,MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30,ranked_0,AlphaFold_benchmark_DMI/random_minimal/distrac...,0.65,237,5,0.02,0.0,...,DEG_APCC_KENBOX_2,None,None,None,A,B,701,937,6,10
5,alphafold3,random_minimal,MDEG_COP1_1_5IGO.DDOC_USP7_MATH_1_3MQS,ranked_0,AlphaFold_benchmark_DMI/random_minimal/reveren...,0.60,143,8,0.05,0.0,...,DEG_COP1_1,None,None,None,A,B,63,205,354,361
10,alphafold3,random_minimal,MDEG_Kelch_Keap1_1_2FLU.DLIG_DLG_GKlike_1_3WP0,ranked_0,AlphaFold_benchmark_DMI/random_minimal/elegant...,0.74,179,6,0.03,0.0,...,DEG_Kelch_Keap1_1,None,None,None,A,B,533,711,77,82
15,alphafold3,random_minimal,MDEG_Kelch_Keap1_2_3WN7.DDOC_USP7_MATH_2_1YY6,ranked_0,AlphaFold_benchmark_DMI/random_minimal/boring_...,0.73,143,7,0.05,0.0,...,DEG_Kelch_Keap1_2,None,None,None,A,B,63,205,26,32
20,alphafold3,random_minimal,MDEG_MDM2_SWIB_1_1YCR.DLIG_PCNA_APIM_2_5MLW,ranked_0,AlphaFold_benchmark_DMI/random_minimal/tender_...,0.83,255,8,0.04,0.0,...,DEG_MDM2_SWIB_1,None,None,None,A,B,1,255,19,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,alphafold3,random_minimal,MTRG_ER_FFAT_1_2RR3.DDOC_MAPK_HePTP_8_2GPH,ranked_0,AlphaFold_benchmark_DMI/random_minimal/exotic_...,0.48,334,9,0.03,0.0,...,TRG_ER_FFAT_1,None,None,None,A,B,21,354,356,364
660,alphafold3,random_minimal,MTRG_LysEnd_GGAAcLL_1_1JWG.DLIG_SUMO_SIM_anti_...,ranked_0,AlphaFold_benchmark_DMI/random_minimal/lonely_...,0.82,78,7,0.08,0.0,...,TRG_LysEnd_GGAAcLL_1,None,None,None,A,B,20,97,7,13
665,alphafold3,random_minimal,MTRG_NES_CRM1_1_3GB8.DDOC_PP1_MyPhoNE_1_1S70,ranked_0,AlphaFold_benchmark_DMI/random_minimal/ridicul...,0.73,299,16,0.05,0.0,...,TRG_NES_CRM1_1,None,None,None,A,B,1,299,2,17
670,alphafold3,random_minimal,MTRG_NLS_Bipartite_1_1PJM.DLIG_PDZ_Class_1_1D5G,ranked_0,AlphaFold_benchmark_DMI/random_minimal/sad_poi...,0.55,96,17,0.15,0.0,...,TRG_NLS_Bipartite_1,None,None,None,A,B,1,96,860,876


In [10]:
print(dataAF[["benchmark_set", "chainA_start", "chainB_start"]])



      benchmark_set chainA_start chainB_start
0    random_minimal          701            6
1    random_minimal          701            6
2    random_minimal          701            6
3    random_minimal          701            6
4    random_minimal          701            6
..              ...          ...          ...
675  random_minimal           50          140
676  random_minimal           50          140
677  random_minimal           50          140
678  random_minimal           50          140
679  random_minimal           50          140

[680 rows x 3 columns]


### 3 RMSD with PyMOL

For all structures, calculate the overall RMSD
- **RMSD_all_atom**: RMSD aligning the whole structure

For DMI, align the domains (chain A) first
- **align_score_domain**: Score of domain alignment
- **num_align_atoms_domain** and **num_align_resi_domain**: Count of aligned atoms/residues of domain
- **RMSD_domain**: RMSD of the domain (chain A) after aligning on the domain
For known_DMI and mutations_DMI (excluding the mutated residues):
- **RMSD_backbone_peptide** and **RMSD_all_atom_peptide**: RMSD of the motif chain (chain B) after aligning on the domain

For DDI perform, use the longest chain (or chain A if both have equal length) as domain and define the shorter one as peptide. Then use the same definition as for DMI

In [11]:
# Calculating the RMSD related values using pymol
import pymol.cmd as cmd


##INITIALIZE DATAFRAME COLUMNS. All values start as None and will be populated during processing.##
dataAF["align_score_domain"] = None
dataAF["num_align_atoms_domain"] = None
dataAF["num_align_resi_domain"] = None
dataAF["RMSD_all_atom"] = None
dataAF["RMSD_domain"] = None
dataAF["RMSD_backbone_peptide"] = None
dataAF["RMSD_all_atom_peptide"] = None

for i,row in dataAF.iterrows():
    benchmark_set = str(row["benchmark_set"])
    _set = "DDI" if "DDI" in benchmark_set else "DMI"
    pdb_id = str(row["PDB_id"]) if row.notnull()["PDB_id"] else None
    pdb_id_2 = str(row["PDB_id_random_paired"]) if row.notnull()["PDB_id_random_paired"] else None
    ddi_pfam_id = None
    ddi_pfam_id_2 = None
    prediction_name = str(row["prediction_name"]) if row.notnull()["prediction_name"] else None
    model_id = str(row["model_id"]) if row.notnull()["model_id"] else None
    chainA_id = str(row["chainA_id"]) if row.notnull()["chainA_id"] else None
    chainB_id = str(row["chainB_id"]) if row.notnull()["chainB_id"] else None
    chainA_start = int(row["chainA_start"]) if row.notnull()["chainA_start"] else None
    chainB_start = int(row["chainB_start"]) if row.notnull()["chainB_start"] else None
    chainA_end = int(row["chainA_end"]) if row.notnull()["chainA_end"] else None
    chainB_end = int(row["chainB_end"]) if row.notnull()["chainB_end"] else None
    chainA_length = int(row["chainA_length"]) if row.notnull()["chainA_length"] else None
    chainB_length = int(row["chainB_length"]) if row.notnull()["chainB_length"] else None

    if model_id == "ranked_0":
        pymol.cmd.reinitialize() 
        print(f"{bcolors.OKBLUE}{prediction_name} ({benchmark_set}){bcolors.ENDC}")    

    structure_path = path_resources / "DMI" / benchmark_set / prediction_name / (model_id + ".pdb")
    
    if not structure_path.exists():
        print(f"\t{bcolors.FAIL}{prediction_name} ({benchmark_set}) does not exist.{bcolors.ENDC} Skip RMSD calculation")
        continue


   
    template_row = dataSolved.loc[np.logical_and(dataSolved["set"] == _set, np.logical_and(dataSolved["PDB_id"] == pdb_id, np.logical_or(dataSolved["DDI_pfam_id"] == ddi_pfam_id, dataSolved["DDI_pfam_id"].isna())))]
    if len(template_row) == 0:
        print(f"\t{bcolors.FAIL}Can't find template structure for {prediction_name} ({benchmark_set}) and PDB ID {pdb_id}.{bcolors.ENDC} Skip RMSD calculation")
        continue
    elif len(template_row) >= 2:
        print(f"\t{bcolors.FAIL}Multiple template structures found for {prediction_name} ({benchmark_set}) and PDB ID {pdb_id}.{bcolors.ENDC} Skip RMSD calculation")
        continue
        
    template_path = path_solved / str(template_row["path"].item())
    

    template2_path = None
    if pdb_id_2 is not None:
        template2_row = dataSolved.loc[np.logical_and(dataSolved["set"] == _set, np.logical_and(dataSolved["PDB_id"] == pdb_id_2, np.logical_or(dataSolved["DDI_pfam_id"] == ddi_pfam_id_2, dataSolved["DDI_pfam_id"].isna())))]
        if len(template2_row) == 0:
            print(f"\t{bcolors.FAIL}Can't find template structure for {prediction_name} ({benchmark_set}) and PDB ID {pdb_id_2}.{bcolors.ENDC} Skip RMSD calculation")
            continue
        elif len(template2_row) >= 2:
            print(f"\t{bcolors.FAIL}Multiple template structures found for {prediction_name} ({benchmark_set}) and PDB ID {pdb_id_2}.{bcolors.ENDC} Skip RMSD calculation")
            continue

        template2_path = path_solved / str(template2_row["path"].item())

    
    #pymol.cmd.reinitialize() # Not needed usually, but slows performance significantly down
    for o in pymol.cmd.get_object_list():
        pymol.cmd.delete("all")
    pymol.cmd.sort()

    # First loading the structures. Use two temporary objects to allow renaming the chains even if the chains have the same name or have switched IDs

    pymol.cmd.load(structure_path, "af")


    if template2_path is None:
        pymol.cmd.load(template2_path, "solvedraw")
        pymol.cmd.create("solved1", f"solvedraw and chain {chainA_id}")
        pymol.cmd.sort()
        pymol.cmd.create("solved2", f"solvedraw and chain {chainB_id}")
        pymol.cmd.delete("solvedraw")

    else:
        pymol.cmd.load(template_path, "solvedraw")
        pymol.cmd.create("solved1", f"solvedraw and chain {chainA_id}")
        pymol.cmd.sort()
        pymol.cmd.create("solved2", f"solvedraw and chain {chainB_id}")
        pymol.cmd.delete("solvedraw")


    
    pymol.cmd.sort()
    # Now rename the chains and create merged object
    pymol.cmd.alter(f"solved1 and chain {chainA_id}", "chain = 'A'")
    pymol.cmd.sort()
    pymol.cmd.alter(f"solved2 and chain {chainB_id}", "chain = 'B'")
    pymol.cmd.sort()
    pymol.cmd.create("solved", f"solved1 or solved2")
    pymol.cmd.delete("solved1")
    pymol.cmd.delete("solved2")
    pymol.cmd.sort()

    # Remove hydrogens and hetatm
    #pymol.cmd.remove(selection="elem 'H' or hetatm")
    pymol.cmd.remove(selection="not backbone and not sidechain or elem 'H'")
    pymol.cmd.sort()

    # Remove alternate location identifiers
    pymol.cmd.remove("not alt ''+A") # Using +A syntax to only effect the atoms with an alternate location identifier set
    pymol.cmd.sort()
    pymol.cmd.alter("all", "alt=''")
    pymol.cmd.sort()




     #Slice the chains to the known start/end residues. For chain B and AF a reindexing is performed as the rms_cur cmd of pymol requires same residue numbers for alignment
    if chainA_start is not None and chainB_start is not None:
        pymol.cmd.create("solved", f"solved and ((chain A and resi {chainA_start}-{chainA_end}) or (chain B and resi {chainB_start}-{chainB_end}))", source_state=0, target_state=0)
        pymol.cmd.sort()
        offsetA = chainA_start - 1
        pymol.cmd.alter("af and chain A", f"resi = (int(resi) + {offsetA})")
        pymol.cmd.sort()

        offsetB = chainB_start - 1
        pymol.cmd.alter("af and chain B", f"resi = (int(resi) + {offsetB})")
        pymol.cmd.sort()
    else:
        print(f"\t{bcolors.FAIL}Can't find information about the chain start/end in the template.{bcolors.ENDC} This may lead to wrong RMSD peptide values, so skip")
        continue

    pymol.cmd.sort()


    # DDI
    chain_align_1, chain_align_2 = "A", "B"
    if _set == "DDI" and chainB_length > chainA_length:
        chain_align_1, chain_align_2 = "B", "A"
        

    #For debugging
    #space = {'solved_resi': [], "af_resi": []}
    #pymol.cmd.iterate("solved and chain B", "solved_resi.append(int(resi))", space=space)
    #pymol.cmd.iterate("af and chain B", "af_resi.append(int(resi))", space=space)

    #    0: RMSD after refinement
    #    1: Number of aligned atoms after refinement
    #    2: Number of refinement cycles
    #    3: RMSD before refinement
    #    4: Number of aligned atoms before refinement
    #    5: Raw alignment score
    #    6: Number of residues aligned
    # Cycles = 0 to prevent rejection of outliers
    
    align_output_1 = pymol.cmd.align(mobile=f"af and chain {chain_align_1}", target=f"solved and chain {chain_align_1}", object="algn_domain", cycles=0)
    pymol.cmd.sort()

    #print(align_output_1)
    #print(f"Full align output: {align_output_1}")
    #print(f"Length: {len(align_output_1)}")
    
    RMSD_domain = align_output_1[0]
    num_align_atoms_domain = align_output_1[1]
    align_score_domain = align_output_1[5]
    num_align_resi_domain = align_output_1[6]

    RMSD_backbone_peptide = pymol.cmd.rms_cur(mobile=f"af and chain {chain_align_1} and bb.", 
                                         target=f"solved and chain {chain_align_1} and bb.", 
                                         object="peptide_super_bb", 
                                         cycles=0 )
                                         #matchmaker=-1)
    #RMSD_all_atom_peptide = pymol.cmd.rms_cur(mobile=f"af and chain {chain_align_2}", target=f"solved and chain {chain_align_2}", object="peptide_super_all_atoms", cycles=0)
    
    #align_output_all = pymol.cmd.align(mobile="af", target="solved", object="algn_all", cycles=0, )
    #RMSD_all_atoms = align_output_all[0]

    dataAF.at[i, "RMSD_domain"] =  RMSD_domain
    dataAF.at[i, "align_score_domain"] =  align_score_domain
    dataAF.at[i, "num_align_atoms_domain"] =  num_align_atoms_domain
    dataAF.at[i, "num_align_resi_domain"] =  num_align_resi_domain

    #if "random" not in benchmark_set:
    dataAF.at[i, "RMSD_backbone_peptide"] =  RMSD_backbone_peptide
    #dataAF.at[i, "RMSD_all_atom_peptide"] =  RMSD_all_atom_peptide

    #dataAF.at[i, "RMSD_all_atom"] =  RMSD_all_atoms
        
display(dataAF)

MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30 (random_minimal)
MDEG_COP1_1_5IGO.DDOC_USP7_MATH_1_3MQS (random_minimal)
MDEG_Kelch_Keap1_1_2FLU.DLIG_DLG_GKlike_1_3WP0 (random_minimal)
MDEG_Kelch_Keap1_2_3WN7.DDOC_USP7_MATH_2_1YY6 (random_minimal)
MDEG_MDM2_SWIB_1_1YCR.DLIG_PCNA_APIM_2_5MLW (random_minimal)
MDEG_SCF_COI1_1_3OGL.DLIG_CaM_IQ_9_2IX7 (random_minimal)
MDEG_SCF_FBXO31_1_5VZU.DLIG_TRAF2_2_1CZY (random_minimal)
MDEG_SCF_TIR1_1_2P1Q.DLIG_ULM_U2AF65_1_1O0P (random_minimal)
MDEG_SIAH_1_2A25.DLIG_CAP-Gly_2_3RDV (random_minimal)
MDEG_SPOP_SBC_1_3HQM.DDOC_MIT_MIM_1_2JQ9 (random_minimal)
MDOC_AGCK_PIF_3_1ATP.DDEG_Kelch_Keap1_1_2FLU (random_minimal)
MDOC_ANK_TNKS_1_3TWU.DLIG_CAP-Gly_1_2PZO (random_minimal)
MDOC_CDC14_PxL_1_6G84.DDOC_PP1_SILK_1_2O8G (random_minimal)
MDOC_CYCLIN_RxL_1_1H25.DLIG_SPRY_1_2JK9 (random_minimal)
MDOC_GSK3_Axin_1_1O9U.DDEG_SPOP_SBC_1_3HQM (random_minimal)
MDOC_MAPK_DCC_7_2B9J.DLIG_ANK_PxLPxL_1_3UXG (random_minimal)
MDOC_MAPK_GRA24_9_5ETA.DLIG_PCNA_APIM_2_5ML

,model_preset,benchmark_set,prediction_name,model_id,model_path,ranking_score,chainA_length,chainB_length,fraction_disordered,has_clash,...,chainA_end,chainB_start,chainB_end,align_score_domain,num_align_atoms_domain,num_align_resi_domain,RMSD_all_atom,RMSD_domain,RMSD_backbone_peptide,RMSD_all_atom_peptide
0,alphafold3,random_minimal,MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30,ranked_0,AlphaFold_benchmark_DMI/random_minimal/distrac...,0.65,237,5,0.02,0.0,...,937,6,10,1224.0,1848,233,None,0.794378,0.494781,None
1,alphafold3,random_minimal,MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30,ranked_1,AlphaFold_benchmark_DMI/random_minimal/distrac...,0.65,237,5,0.02,0.0,...,937,6,10,1224.0,1848,233,None,0.673389,0.498352,None
2,alphafold3,random_minimal,MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30,ranked_2,AlphaFold_benchmark_DMI/random_minimal/distrac...,0.65,237,5,0.02,0.0,...,937,6,10,1224.0,1848,233,None,0.676211,0.467665,None
3,alphafold3,random_minimal,MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30,ranked_3,AlphaFold_benchmark_DMI/random_minimal/distrac...,0.64,237,5,0.02,0.0,...,937,6,10,1224.0,1848,233,None,0.657313,0.441906,None
4,alphafold3,random_minimal,MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30,ranked_4,AlphaFold_benchmark_DMI/random_minimal/distrac...,0.62,237,5,0.02,0.0,...,937,6,10,1224.0,1848,233,None,0.724507,0.454487,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,alphafold3,random_minimal,MTRG_PTS1_2C0L.DLIG_WD40_WDR5_WIN_2_4CY3,ranked_0,AlphaFold_benchmark_DMI/random_minimal/gloomy_...,0.93,312,4,0.02,0.0,...,361,140,143,1624.0,2356,303,None,0.684738,0.364558,None
676,alphafold3,random_minimal,MTRG_PTS1_2C0L.DLIG_WD40_WDR5_WIN_2_4CY3,ranked_1,AlphaFold_benchmark_DMI/random_minimal/gloomy_...,0.92,312,4,0.01,0.0,...,361,140,143,1624.0,2356,303,None,0.684467,0.338946,None
677,alphafold3,random_minimal,MTRG_PTS1_2C0L.DLIG_WD40_WDR5_WIN_2_4CY3,ranked_2,AlphaFold_benchmark_DMI/random_minimal/gloomy_...,0.92,312,4,0.02,0.0,...,361,140,143,1624.0,2356,303,None,1.018179,0.672213,None
678,alphafold3,random_minimal,MTRG_PTS1_2C0L.DLIG_WD40_WDR5_WIN_2_4CY3,ranked_3,AlphaFold_benchmark_DMI/random_minimal/gloomy_...,0.91,312,4,0.01,0.0,...,361,140,143,1624.0,2356,303,None,0.658395,0.313743,None


In [12]:
# Step 1: Filter only the ranked_0 rows
ranked_0_only = dataAF[dataAF['model'].str.contains('ranked_0')]

# Step 2: From those, filter rows where RMSD > 5.0
high_rmsd_rows = ranked_0_only[ranked_0_only['RMSD_backbone_peptide'] > 5.0]

# Step 3: Print results
print(f"Found {len(high_rmsd_rows)} rows with RMSD > 5.0 in ranked_0 models")
print(high_rmsd_rows)


Found 45 rows with RMSD > 5.0
    model_preset   benchmark_set  \
25    alphafold3  random_minimal   
26    alphafold3  random_minimal   
27    alphafold3  random_minimal   
28    alphafold3  random_minimal   
29    alphafold3  random_minimal   
90    alphafold3  random_minimal   
91    alphafold3  random_minimal   
92    alphafold3  random_minimal   
93    alphafold3  random_minimal   
94    alphafold3  random_minimal   
165   alphafold3  random_minimal   
166   alphafold3  random_minimal   
167   alphafold3  random_minimal   
168   alphafold3  random_minimal   
169   alphafold3  random_minimal   
194   alphafold3  random_minimal   
395   alphafold3  random_minimal   
396   alphafold3  random_minimal   
397   alphafold3  random_minimal   
398   alphafold3  random_minimal   
399   alphafold3  random_minimal   
410   alphafold3  random_minimal   
411   alphafold3  random_minimal   
412   alphafold3  random_minimal   
413   alphafold3  random_minimal   
414   alphafold3  random_minimal  

In [13]:
two_rmsd_rows = dataAF[dataAF['RMSD_backbone_peptide'] < 2.0]
print(f"Found {len(two_rmsd_rows)} rows with RMSD < 2.0")

Found 478 rows with RMSD < 2.0


In [14]:
print(dataAF)

    model_preset   benchmark_set  \
0     alphafold3  random_minimal   
1     alphafold3  random_minimal   
2     alphafold3  random_minimal   
3     alphafold3  random_minimal   
4     alphafold3  random_minimal   
..           ...             ...   
675   alphafold3  random_minimal   
676   alphafold3  random_minimal   
677   alphafold3  random_minimal   
678   alphafold3  random_minimal   
679   alphafold3  random_minimal   

                                       prediction_name  model_id  \
0    MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30  ranked_0   
1    MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30  ranked_1   
2    MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30  ranked_2   
3    MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30  ranked_3   
4    MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30  ranked_4   
..                                                 ...       ...   
675           MTRG_PTS1_2C0L.DLIG_WD40_WDR5_WIN_2_4CY3  ranked_0   
676           MTRG_PTS1

### 4 DockQ
Calculate DockQ metrics for the known_DMI and known_DDI set using the offical package. The following columns are added to the table:
- **DockQ**: DockQ metric
- **iRMSD**: RMSD of interfacial residues
- **LRMSD**: Ligand RMSD. In case of DMI is this the motif, for DDI the smaller domain
- **Fnonnat**: Fraction of predicted contacts that are not native (same as FPR)

For more details read the details on the offical GitHub repo: [https://github.com/bjornwallner/DockQ](https://github.com/bjornwallner/DockQ)

In [24]:
from DockQ.DockQ import load_PDB, run_on_all_native_interfaces

dataAF["DockQ"] = np.nan
dataAF["iRMSD"] = np.nan
dataAF["LRMSD"] = np.nan
dataAF["Fnonnat"] = np.nan
for i, row in dataAF[dataAF["benchmark_set"].isin(["known_extension"])].iterrows():
    benchmark_set = str(row["benchmark_set"])
    _set = "DDI" if "DDI" in benchmark_set else "DMI"
    pdb_id = str(row["PDB_id"]) if row.notnull()["PDB_id"] else None
    pdb_id_2 =  None
    ddi_pfam_id =  None
    ddi_pfam_id_2 = None
    prediction_name = str(row["prediction_name"]) if row.notnull()["prediction_name"] else None
    model_id = str(row["model_id"]) if row.notnull()["model_id"] else None
    chainA_id = str(row["chainA_id"]) if row.notnull()["chainA_id"] else None
    chainB_id = str(row["chainB_id"]) if row.notnull()["chainB_id"] else None
    chainA_start = int(row["chainA_start"]) if row.notnull()["chainA_start"] else None
    chainB_start = int(row["chainB_start"]) if row.notnull()["chainB_start"] else None
    chainA_end = int(row["chainA_end"]) if row.notnull()["chainA_end"] else None
    chainB_end = int(row["chainB_end"]) if row.notnull()["chainB_end"] else None

    if model_id == "ranked_0":
        print(f"{bcolors.OKBLUE}{prediction_name} ({benchmark_set}){bcolors.ENDC}")

    structure_path = path_resources / "DMI" / benchmark_set / prediction_name / (model_id + ".pdb")
    if not structure_path.exists():
        print(f"\t{bcolors.FAIL}{prediction_name} ({benchmark_set}) does not exist.{bcolors.ENDC} Skip DockQ")
        continue

    template_row = dataSolved.loc[np.logical_and(dataSolved["set"] == _set, np.logical_and(dataSolved["PDB_id"] == pdb_id, np.logical_or(dataSolved["DDI_pfam_id"] == ddi_pfam_id, dataSolved["DDI_pfam_id"].isna())))]
    if len(template_row) == 0:
        print(f"\t{bcolors.FAIL}Can't find template structure for {prediction_name} ({benchmark_set}) and PDB ID {pdb_id}.{bcolors.ENDC} Skip")
        continue
    elif len(template_row) >= 2:
        print(f"\t{bcolors.FAIL}Multiple template structures found for {prediction_name} ({benchmark_set}) and PDB ID {pdb_id}.{bcolors.ENDC} Skip")
        continue
    template_path = path_solved / str(template_row["path"].item())
    dockq_structure_af = load_PDB(str(structure_path))
    dockq_structure_solved = load_PDB(str(template_path))

    chain_map = {chainA_id: "A", chainB_id:"B"}
    chain_key = chainA_id + chainB_id

    result = run_on_all_native_interfaces(dockq_structure_af, dockq_structure_solved, chain_map=chain_map)[0]
    dataAF.at[i, "DockQ"] = result[chain_key]["DockQ"]
    dataAF.at[i, "iRMSD"] = result[chain_key]["iRMSD"]
    dataAF.at[i, "LRMSD"] = result[chain_key]["LRMSD"]
    dataAF.at[i, "Fnonnat"] = np.float64(result[chain_key]["fnonnat"])

display(dataAF)


,model_preset,benchmark_set,prediction_name,model_id,model_path,ranking_score,chainA_length,chainB_length,fraction_disordered,has_clash,...,num_align_atoms_domain,num_align_resi_domain,RMSD_all_atom,RMSD_domain,RMSD_backbone_peptide,RMSD_all_atom_peptide,DockQ,iRMSD,LRMSD,Fnonnat
0,alphafold3,random_minimal,MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30,ranked_0,AlphaFold_benchmark_DMI/random_minimal/distrac...,0.65,237,5,0.02,0.0,...,1848,233,10.938316,0.794378,77.63578,76.997147,NaN,NaN,NaN,NaN
1,alphafold3,random_minimal,MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30,ranked_1,AlphaFold_benchmark_DMI/random_minimal/distrac...,0.65,237,5,0.02,0.0,...,1848,233,10.788113,0.673389,76.731873,75.978645,NaN,NaN,NaN,NaN
2,alphafold3,random_minimal,MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30,ranked_2,AlphaFold_benchmark_DMI/random_minimal/distrac...,0.65,237,5,0.02,0.0,...,1848,233,10.806915,0.676211,76.831932,76.127953,NaN,NaN,NaN,NaN
3,alphafold3,random_minimal,MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30,ranked_3,AlphaFold_benchmark_DMI/random_minimal/distrac...,0.64,237,5,0.02,0.0,...,1848,233,10.832501,0.657313,77.142761,76.315002,NaN,NaN,NaN,NaN
4,alphafold3,random_minimal,MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30,ranked_4,AlphaFold_benchmark_DMI/random_minimal/distrac...,0.62,237,5,0.02,0.0,...,1848,233,10.788817,0.724507,76.870415,75.98243,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,alphafold3,random_minimal,MTRG_PTS1_2C0L.DLIG_WD40_WDR5_WIN_2_4CY3,ranked_0,AlphaFold_benchmark_DMI/random_minimal/gloomy_...,0.93,312,4,0.02,0.0,...,2356,303,7.710459,0.684738,69.098969,68.151733,NaN,NaN,NaN,NaN
676,alphafold3,random_minimal,MTRG_PTS1_2C0L.DLIG_WD40_WDR5_WIN_2_4CY3,ranked_1,AlphaFold_benchmark_DMI/random_minimal/gloomy_...,0.92,312,4,0.01,0.0,...,2356,303,7.707559,0.684467,69.105652,68.11792,NaN,NaN,NaN,NaN
677,alphafold3,random_minimal,MTRG_PTS1_2C0L.DLIG_WD40_WDR5_WIN_2_4CY3,ranked_2,AlphaFold_benchmark_DMI/random_minimal/gloomy_...,0.92,312,4,0.02,0.0,...,2356,303,7.750147,1.018179,69.197678,68.172096,NaN,NaN,NaN,NaN
678,alphafold3,random_minimal,MTRG_PTS1_2C0L.DLIG_WD40_WDR5_WIN_2_4CY3,ranked_3,AlphaFold_benchmark_DMI/random_minimal/gloomy_...,0.91,312,4,0.01,0.0,...,2356,303,7.704717,0.658395,69.042198,68.116119,NaN,NaN,NaN,NaN


### 5 IPSAE metric
Calculating the new ipSAE metric using the file from the repo ([https://github.com/DunbrackLab/IPSAE/blob/main/ipsae.py](https://github.com/DunbrackLab/IPSAE/blob/main/ipsae.py)). Currently, only AlphaFold 3 is possible as the files from John do not include the confidence.json files anymore. Adds the following column:
- **ipSAE** (all structures): The ipSAE metric for the AlphaFold prediction

In [49]:
def calc_ipsae_metric(row: pd.Series):
    path_cif = path_AF_luck_drive / Path(row["model_path"])

    sample_dir = path_cif.parent
    design_name = sample_dir.parent.name
    sample_name = sample_dir.name
    base_filename = f"{design_name}_{sample_name}"
    path_confidences = sample_dir / f"{base_filename}_confidences.json"
    path_summary_confidences = sample_dir / f"{base_filename}_summary_confidences.json"


    with tempfile.TemporaryDirectory() as tmpdir:
        shutil.copy(path_cif, tmp_path_cif := (Path(tmpdir) / "model.cif"))
        shutil.copy(path_confidences, tmp_path_confidences := (Path(tmpdir) / "confidences.json"))
        shutil.copy(path_summary_confidences, tmp_path_summary_confidences := (Path(tmpdir) / "summary_confidences.json"))
        subprocess.run(["python", path_ipsae_script, tmp_path_confidences, tmp_path_cif, "10", "10"], env=os.environ.copy())

        path_output = Path(tmpdir) / "model_10_10.txt"

        df_ipsae = pd.read_csv(path_output, header=0, skiprows=[0], sep=" ", skipinitialspace=True)
    return df_ipsae

# For AF2 the json files do not exist anymore
if af_mode == "AF3":
    dataAF["ipSAE"] = np.nan
    for i, row in dataAF.iterrows():
        if row["model_id"] == "ranked_0":
            print(row["prediction_name"], f"({round(100*i/len(dataAF))} %)")
        df_ipsae = calc_ipsae_metric(row)
        dataAF.at[i, "ipSAE"] = np.float64(df_ipsae["ipSAE"][2])
display(dataAF["ipSAE"])

DEG_APCC_KENBOX_2_M15_M39_D1_D499 (96 %)
DEG_APCC_KENBOX_2_M15_M39_Dmin (96 %)
DEG_APCC_KENBOX_2_M1_M189_D1_D499 (96 %)
DEG_APCC_KENBOX_2_M1_M189_Dmin (97 %)
DEG_APCC_KENBOX_2_M1_M57_D1_D499 (97 %)
DEG_APCC_KENBOX_2_M1_M57_Dmin (97 %)
DEG_APCC_KENBOX_2_M20_M34_D1_D499 (97 %)
DEG_APCC_KENBOX_2_M20_M34_Dmin (97 %)
DEG_APCC_KENBOX_2_M5_M49_D1_D499 (98 %)
DEG_APCC_KENBOX_2_M5_M49_Dmin (98 %)
DEG_APCC_KENBOX_2_MFL_Dmin (98 %)
DEG_APCC_KENBOX_2_Mmin_D1_D499 (98 %)
DEG_APCC_KENBOX_2_Mmin_DFL (98 %)
DEG_Kelch_Keap1_1_M1_M568_D175_D624 (98 %)
DEG_Kelch_Keap1_1_M1_M568_D295_D624 (99 %)
DEG_Kelch_Keap1_1_M1_M568_Dmin (99 %)
DEG_Kelch_Keap1_1_M41_M118_D175_D624 (99 %)
DEG_Kelch_Keap1_1_M41_M118_D295_D624 (99 %)
DEG_Kelch_Keap1_1_M41_M118_Dmin (99 %)
DEG_Kelch_Keap1_1_M53_M106_D175_D624 (99 %)
DEG_Kelch_Keap1_1_M53_M106_D295_D624 (100 %)
DEG_Kelch_Keap1_1_M53_M106_Dmin (100 %)
DEG_Kelch_Keap1_1_M65_M94_D175_D624 (100 %)
DEG_Kelch_Keap1_1_M65_M94_D295_D624 (100 %)
DEG_Kelch_Keap1_1_M65_M94_Dmin (100

Traceback (most recent call last):
  File "/Users/imb/Desktop/ipsae.py", line 416, in <module>
    data = json.load(file)
  File "/opt/miniconda3/envs/pymol_env/lib/python3.10/json/__init__.py", line 293, in load
    return loads(fp.read(),
  File "/opt/miniconda3/envs/pymol_env/lib/python3.10/codecs.py", line 322, in decode
    (result, consumed) = self._buffer_decode(data, self.errors, final)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd0 in position 25165832: invalid continuation byte


EmptyDataError: No columns to parse from file

In [51]:
print(tempfile.TemporaryDirectory())

<TemporaryDirectory '/var/folders/y7/rw09j59914xbqkvqvgq68_1c0000gn/T/tmpwe2zj6hd'>


### 6 Interaction metrics
Calculate the interaction interface metrics with the newly developed library. Adds the following columns to all structures with an experimentally solved counterpart (None if not):
- **min_distance** [Å]: Distance of the C_alpha atoms (in Angstroms) in the interface if at least one atom pair between the residues is closer than 15 Angstrom. 
- **buried_area** [Å²]: Buried surface area of the interface (in Angstrom squared)
- **disulfide_bonds** [count]: Number of disulfide bonds in the interface
- **salt_bridges** [count]: Number of salt bridges in the interface
- **hbonds** [count]: Number of hydrogen bonds in the interface
- **hydrophobic_interactions** [count]: Number of hydrophobic interactions in the interface

For a detailed explanation see either the library file (measure_PPI.py) or the bachelor thesis (can be found in the group drive)

In [17]:
libpath = Path("../Interface Metrics").resolve()
print(libpath)
sys.path.insert(0, str(libpath))
import measure_PPI

/Users/imb/Interface Metrics


ModuleNotFoundError: No module named 'measure_PPI'

In [8]:
pathObj = []
for i, row in dataAF.iterrows():
    benchmark_set = str(row["benchmark_set"])
    _set = "DDI" if "DDI" in benchmark_set else "DMI"
    prediction_name = str(row["prediction_name"]) if row.notnull()["prediction_name"] else None
    model_id = str(row["model_id"]) if row.notnull()["model_id"] else None

    structure_path = path_AF / _set / benchmark_set / prediction_name / (model_id + ".pdb")
    if not structure_path.exists():
        if row["model_id"] == "ranked_0":
            print(f"\t{bcolors.FAIL}{prediction_name} ({benchmark_set}) does not exist.{bcolors.ENDC} Skip interface metrics")
        continue

    pathObj.append((structure_path.resolve(), prediction_name))
df_intf_metrics = measure_PPI.Run(pathObj=pathObj, num_threads=12)

	PF07724_PF00227_1OFH_C_resi39_resi340.H_resi1_resi172 (known_DDI) does not exist. Skip interface metrics
	PF14978_PF00327_3J7Y_o_resi13_resi101.Z_resi57_resi127 (known_DDI) does not exist. Skip interface metrics
[2025-05-15 19:06:35,957 | measure_PPI | INFO] Started Taskpool of 12 processes for 3170 files
[2025-05-15 19:06:41,001 | measure_PPI | INFO] 2% - ETA 0:03:43 | current speed 14.077 s⁻¹ | average speed 13.878 s⁻¹
[2025-05-15 19:06:46,038 | measure_PPI | INFO] 5% - ETA 0:02:40 | current speed 23.227 s⁻¹ | average speed 18.55 s⁻¹
[2025-05-15 19:06:51,048 | measure_PPI | INFO] 11% - ETA 0:01:53 | current speed 36.922 s⁻¹ | average speed 24.649 s⁻¹
[2025-05-15 19:06:56,112 | measure_PPI | INFO] 17% - ETA 0:01:37 | current speed 34.174 s⁻¹ | average speed 27.042 s⁻¹
[2025-05-15 19:07:01,129 | measure_PPI | INFO] 22% - ETA 0:01:27 | current speed 33.074 s⁻¹ | average speed 28.245 s⁻¹
[2025-05-15 19:07:06,139 | measure_PPI | INFO] 27% - ETA 0:01:20 | current speed 31.148 s⁻¹ | averag

In [9]:
display(df_intf_metrics)

,structure_name,file,hbonds,salt_bridges,buried_area,min_distance,hydrophobic_interactions,disulfide_bonds
2924,D1PF00009_PF01873_2D74.D2PF00026_PF06394_1F34,ranked_0.pdb,1,0,1692.652,4.191,86,0
2925,D1PF00009_PF01873_2D74.D2PF00026_PF06394_1F34,ranked_1.pdb,3,0,1708.317,3.670,96,0
2933,D1PF00009_PF01873_2D74.D2PF00026_PF06394_1F34,ranked_2.pdb,1,1,1907.898,4.643,79,0
2932,D1PF00009_PF01873_2D74.D2PF00026_PF06394_1F34,ranked_3.pdb,5,0,1577.972,3.718,107,0
2934,D1PF00009_PF01873_2D74.D2PF00026_PF06394_1F34,ranked_4.pdb,15,1,2139.539,3.955,101,0
...,...,...,...,...,...,...,...,...
2695,TRG_PTS1_2C0L_NAKL.NAKD,ranked_0.pdb,10,3,839.162,4.782,11,0
2698,TRG_PTS1_2C0L_NAKL.NAKD,ranked_1.pdb,8,3,862.629,4.900,10,0
2696,TRG_PTS1_2C0L_NAKL.NAKD,ranked_2.pdb,12,2,861.115,4.704,9,0
2699,TRG_PTS1_2C0L_NAKL.NAKD,ranked_3.pdb,11,2,854.038,4.943,9,0


In [11]:
dataAF["min_distance"] = None
dataAF["buried_area"] = None
dataAF["disulfide_bonds"] = None
dataAF["salt_bridges"] = None
dataAF["hbonds"] = None
dataAF["hydrophobic_interactions"] = None

for i, row in dataAF.iterrows():
    row_intf = df_intf_metrics[(df_intf_metrics["structure_name"] == row["prediction_name"]) & (df_intf_metrics["file"] == row["model_id"] + ".pdb")]
    if len(row_intf) != 1:
        print(f"\t{bcolors.FAIL}Failed to locate {row["prediction_name"]} {row["model_id"]}{bcolors.ENDC}")
        continue

    dataAF.at[i, "buried_area"] = row_intf["buried_area"].item()
    dataAF.at[i, "min_distance"] = row_intf["min_distance"].item()
    dataAF.at[i, "salt_bridges"] = row_intf["salt_bridges"].item()
    dataAF.at[i, "hbonds"] = row_intf["hbonds"].item()
    dataAF.at[i, "hydrophobic_interactions"] = row_intf["hydrophobic_interactions"].item()
    dataAF.at[i, "disulfide_bonds"] = row_intf["disulfide_bonds"].item()
display(dataAF)

	Failed to locate PF07724_PF00227_1OFH_C_resi39_resi340.H_resi1_resi172 ranked_0
	Failed to locate PF07724_PF00227_1OFH_C_resi39_resi340.H_resi1_resi172 ranked_1
	Failed to locate PF07724_PF00227_1OFH_C_resi39_resi340.H_resi1_resi172 ranked_2
	Failed to locate PF07724_PF00227_1OFH_C_resi39_resi340.H_resi1_resi172 ranked_3
	Failed to locate PF07724_PF00227_1OFH_C_resi39_resi340.H_resi1_resi172 ranked_4
	Failed to locate PF14978_PF00327_3J7Y_o_resi13_resi101.Z_resi57_resi127 ranked_0
	Failed to locate PF14978_PF00327_3J7Y_o_resi13_resi101.Z_resi57_resi127 ranked_1
	Failed to locate PF14978_PF00327_3J7Y_o_resi13_resi101.Z_resi57_resi127 ranked_2
	Failed to locate PF14978_PF00327_3J7Y_o_resi13_resi101.Z_resi57_resi127 ranked_3
	Failed to locate PF14978_PF00327_3J7Y_o_resi13_resi101.Z_resi57_resi127 ranked_4


,project_name,run_id,benchmark_set,prediction_name,model_id,chainA_length,chainB_length,chainA_id,chainB_id,chainA_start,...,DockQ,iRMSD,LRMSD,Fnonnat,buried_area,min_distance,disulfide_bonds,salt_bridges,hbonds,hydrophobic_interactions
0,AlphaFold_benchmark,run37,known_DMI,DEG_APCC_KENBOX_2_4GGD,ranked_0,312,5,A,B,165,...,0.878344,0.603831,1.575394,0.086957,613.651,6.063,0,0,9,0
1,AlphaFold_benchmark,run37,known_DMI,DEG_APCC_KENBOX_2_4GGD,ranked_1,312,5,A,B,165,...,0.880716,0.418230,1.100588,0.050000,580.31,6.083,0,0,9,0
2,AlphaFold_benchmark,run37,known_DMI,DEG_APCC_KENBOX_2_4GGD,ranked_2,312,5,A,B,165,...,0.883186,0.641834,1.776257,0.185185,662.104,6.072,0,0,10,3
3,AlphaFold_benchmark,run37,known_DMI,DEG_APCC_KENBOX_2_4GGD,ranked_3,312,5,A,B,165,...,0.475511,1.686332,5.358800,0.363636,398.498,5.417,0,0,2,0
4,AlphaFold_benchmark,run37,known_DMI,DEG_APCC_KENBOX_2_4GGD,ranked_4,312,5,A,B,165,...,0.223400,2.928606,9.908745,0.888889,323.304,5.092,0,0,2,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3175,AlphaFold_benchmark_DDI,run6,random_DDI,D1PF18773_PF00071_2X19.D2PF00009_PF01873_2D74,ranked_0,60,113,B,B,392,...,NaN,NaN,NaN,NaN,1617.382,5.591,0,3,7,56
3176,AlphaFold_benchmark_DDI,run6,random_DDI,D1PF18773_PF00071_2X19.D2PF00009_PF01873_2D74,ranked_1,60,113,B,B,392,...,NaN,NaN,NaN,NaN,791.256,6.373,0,0,3,7
3177,AlphaFold_benchmark_DDI,run6,random_DDI,D1PF18773_PF00071_2X19.D2PF00009_PF01873_2D74,ranked_2,60,113,B,B,392,...,NaN,NaN,NaN,NaN,882.547,7.906,0,1,2,11
3178,AlphaFold_benchmark_DDI,run6,random_DDI,D1PF18773_PF00071_2X19.D2PF00009_PF01873_2D74,ranked_3,60,113,B,B,392,...,NaN,NaN,NaN,NaN,1020.896,4.628,0,3,7,44


### Export table

In [15]:
# Inspect
enhance_dataframe()
dataAF

Column num_mutations not (yet) in data frame
Column hbonds not (yet) in data frame
Column salt_bridges not (yet) in data frame
Column hydrophobic_interactions not (yet) in data frame
Column disulfide_bonds not (yet) in data frame
Column num_mutations not (yet) in data frame
Column buried_area not (yet) in data frame
Column min_distance not (yet) in data frame
Column disulfide_bonds not (yet) in data frame
Column salt_bridges not (yet) in data frame
Column hbonds not (yet) in data frame
Column hydrophobic_interactions not (yet) in data frame


,model_preset,benchmark_set,prediction_name,model_id,model_path,ranking_score,chainA_length,chainB_length,chainA_id,chainB_id,...,align_score_domain,num_align_atoms_domain,num_align_resi_domain,RMSD_domain,RMSD_backbone_peptide,RMSD_all_atom_peptide,RMSD_all_atom,iPAE,pDockQ,chains_flipped
0,alphafold3,random_minimal,MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30,ranked_0,AlphaFold_benchmark_DMI/random_minimal/distrac...,0.65,237,5,A,B,...,1224.0,1848,233,0.794378,0.494781,None,None,10.60,0.07,True
1,alphafold3,random_minimal,MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30,ranked_1,AlphaFold_benchmark_DMI/random_minimal/distrac...,0.65,237,5,A,B,...,1224.0,1848,233,0.673389,0.498352,None,None,11.00,0.09,True
2,alphafold3,random_minimal,MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30,ranked_2,AlphaFold_benchmark_DMI/random_minimal/distrac...,0.65,237,5,A,B,...,1224.0,1848,233,0.676211,0.467665,None,None,10.80,0.08,True
3,alphafold3,random_minimal,MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30,ranked_3,AlphaFold_benchmark_DMI/random_minimal/distrac...,0.64,237,5,A,B,...,1224.0,1848,233,0.657313,0.441906,None,None,11.80,0.08,True
4,alphafold3,random_minimal,MDEG_APCC_KENBOX_2_4GGD.DTRG_AP2beta_CARGO_1_2G30,ranked_4,AlphaFold_benchmark_DMI/random_minimal/distrac...,0.62,237,5,A,B,...,1224.0,1848,233,0.724507,0.454487,None,None,11.90,0.08,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,alphafold3,random_minimal,MTRG_PTS1_2C0L.DLIG_WD40_WDR5_WIN_2_4CY3,ranked_0,AlphaFold_benchmark_DMI/random_minimal/gloomy_...,0.93,312,4,A,B,...,1624.0,2356,303,0.684738,0.364558,None,None,3.50,0.31,True
676,alphafold3,random_minimal,MTRG_PTS1_2C0L.DLIG_WD40_WDR5_WIN_2_4CY3,ranked_1,AlphaFold_benchmark_DMI/random_minimal/gloomy_...,0.92,312,4,A,B,...,1624.0,2356,303,0.684467,0.338946,None,None,3.60,0.31,True
677,alphafold3,random_minimal,MTRG_PTS1_2C0L.DLIG_WD40_WDR5_WIN_2_4CY3,ranked_2,AlphaFold_benchmark_DMI/random_minimal/gloomy_...,0.92,312,4,A,B,...,1624.0,2356,303,1.018179,0.672213,None,None,3.65,0.30,True
678,alphafold3,random_minimal,MTRG_PTS1_2C0L.DLIG_WD40_WDR5_WIN_2_4CY3,ranked_3,AlphaFold_benchmark_DMI/random_minimal/gloomy_...,0.91,312,4,A,B,...,1624.0,2356,303,0.658395,0.313743,None,None,3.60,0.30,True


In [16]:
# Save
enhance_dataframe()
dataAF.to_csv(path_AF / (path_AF.name + "_metrics_only_random_minimal.tsv"), sep="\t", index=None)
dataAF.to_excel(path_AF/ (path_AF.name + "_metrics_only_random_minimal.xlsx"), sheet_name=f"{af_mode} metrics", index=None)

Column num_mutations not (yet) in data frame
Column hbonds not (yet) in data frame
Column salt_bridges not (yet) in data frame
Column hydrophobic_interactions not (yet) in data frame
Column disulfide_bonds not (yet) in data frame
Column num_mutations not (yet) in data frame
Column buried_area not (yet) in data frame
Column min_distance not (yet) in data frame
Column disulfide_bonds not (yet) in data frame
Column salt_bridges not (yet) in data frame
Column hbonds not (yet) in data frame
Column hydrophobic_interactions not (yet) in data frame
